In [1]:
import numpy as np
#import cv2

from multiprocessing.pool import ThreadPool
from collections import deque
import matplotlib.pyplot as plt
import json
########### Python 2.7 #############
import httplib, urllib, base64
import time
import sys
sys.path.append("../lib")
from load import getValData
from load import faces
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
from six.moves import cPickle
from numpy import genfromtxt


Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, CuDNN 3007)


In [ ]:
trX, teX, trY, teY = faces(onehot=True)

# ValX , ValY = getValData()
teX = trX.reshape(-1, 1, 48, 48)
# ValX = ValX.reshape(-1, 1, 48, 48)

In [ ]:
#teX = genfromtxt('../CKPlus/CKfaces.csv', delimiter=',')
#teX = teX.reshape(-1, 1, 48, 48)

In [ ]:
teX = teX[17361:]
print teX.shape

In [ ]:
#teY = genfromtxt('../CKPlus/labels.csv', delimiter=',')
print teY.shape

In [ ]:
headers = {
    # Request headers
    'Content-Type': 'application/octet-stream',
    'Ocp-Apim-Subscription-Key': '0a262268ddda47fab90091594d36936a',
}

params = urllib.urlencode({
    # Request parameters
    'faceRectangles': '{string}',
})

imgPath = "../Data/1450656001/faces/1290435672271888384/112.jpg"
payload = "{ 'url': 'www.inf.kcl.ac.uk/pg/sagarj/images/Sagar_Pic.jpg' }"

mimeString = "data:image/jpg;base64,"

scoreFile = "../Logs/MsftTrainScores3.pkl"



In [ ]:
def getPayload(filePath):
    with open(filePath, "rb") as imageFile:
        f = imageFile.read()
        b = bytearray(f)
    return b

def makeEmoRequest(payload):
    data = {}
    try:
        conn = httplib.HTTPSConnection('api.projectoxford.ai')
        conn.request("POST", "/emotion/v1.0/recognize?", payload , headers)
        response = conn.getresponse()
        data = json.loads(response.read())
        print response.status, response.reason
        conn.close()
    except Exception as e:
        print e
        return None
    return data

0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

In [ ]:
for i in range(len(teX)):
    plt.imshow(teX[i][0],cmap = cm.Greys_r)
    plt.savefig('tmp')
    time.sleep(2)
    image = getPayload("tmp.png")
    time.sleep(8)
    data = makeEmoRequest(image)
    
    time.sleep(2)
    if len(data) == 0:
        logline = str(0.143) + "," + str(0.143) + "," + str(0.143) + \
        "," + str(0.143) + "," + str(0.143) + "," + str(0.143) + \
        "," + str(0.143)
    else:
        face = data[0]
        logline = str(face['scores']['anger']) + "," + str(face['scores']['disgust']) + "," + str(face['scores']['fear']) + \
        "," + str(face['scores']['happiness']) + "," + str(face['scores']['sadness']) + "," + str(face['scores']['surprise']) + \
        "," + str(face['scores']['neutral'])
    
    print logline   
    f = open(scoreFile, 'a+')
    cPickle.dump(logline , f);
    f.close()
    os.remove("tmp.png")

In [ ]:
print i